<a href="https://colab.research.google.com/github/charlotter62/EU-ETS-EUTL/blob/main/O1_xml_operators_byaccountID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
!pip install wget
import wget

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
from socket import error as SocketError
import errno

In [ ]:
def isXML(filepath):
  f = open(filepath, "r")
  xml=False
  if "<?xml" in f.readline():
    xml=True
  return(xml)

In [ ]:
def patientDownload(link,savename):
  success=0
  while(success!=1):
    try:
      wget.download(link,savename)
      if(isXML(savename)):
        success=1
      else:
        print("Download failed not XML, attempting again")
        os.remove(savename)
        time.sleep(10)
    except SocketError as e:
      if e.errno != errno.ECONNRESET:
          raise # Not error we are looking for
      print("Download failed on server, attempting again")
      time.sleep(10)
    # except ConnectionResetError as e:
    #   print("Download failed on server, attempting again")
    #   time.sleep(10)

In [ ]:
def checkDownloadError(link):
  page = requests.get(link)
  soup = BeautifulSoup(page.content, "html.parser")
  error = soup.find("pre", {"class":"errortext"})
  if error:
    error=error.string
  else:
    error="No error"
  return(error)

In [ ]:
def getXMLbyAccountID(id,regcode):
  savename = workingdir+"XML files/"+ regcode+"_"+str(id)+"_operator.xml"
  if(os.path.exists(savename)):
    print("File already exists! " + savename)
  else:
    #link = "https://ec.europa.eu/clima/ets/exportEntry.do?form=ohaDetails&installationName=&permitIdentifier=&searchType=&mainActivityType=&accountType=&selectedPeriods=&complianceStatus=&languageCode=en&account.registryCode="+regcode+"&accountStatus=&accountID="+str(id)+"&accountHolder=&registryCode="+regcode+"&installationIdentifier=&action=all&primaryAuthRep=&identifierInReg=&returnURL=installationName%3D%26accountHolder%3D%26search%3DSearch%26permitIdentifier%3D%26form%3Doha%26searchType%3Doha%26mainActivityType%3D-1%26currentSortSettings%3D%26installationIdentifier%3D%26languageCode%3Den&buttonAction="+"&exportType=1&exportAction=accountAll&exportOK=exportOK"
    link = "https://ec.europa.eu/clima/ets/exportEntry.do?form=ohaDetails&installationName=&permitIdentifier=&searchType=&mainActivityType=&accountType=&selectedPeriods=&complianceStatus=&languageCode=en&account.registryCode="+regcode+"&accountStatus=&accountID="+str(id)+"&accountHolder=&registryCode="+regcode+"&installationIdentifier=&action=all&primaryAuthRep=&identifierInReg=&returnURL=installationName%3D%26accountHolder%3D%26search%3DSearch%26permitIdentifier%3D%26form%3Doha%26searchType%3Doha%26mainActivityType%3D-1%26currentSortSettings%3D%26installationIdentifier%3D%26languageCode%3Den&buttonAction="+"&exportType=1&exportAction=accountAll&exportOK=exportOK"
    print(link)
    if("No results found" in checkDownloadError(link)):
      print("Not operator account!")
    else:
      print("Downloading..." + savename)
      patientDownload(link,savename)

Single test...

In [ ]:
workingdir = "/gdrive/MyDrive/Brookings/XML_downloads/xml-operators-byaccountID/"
ID = int(101987)
getXMLbyAccountID(ID,"NO")

Downloading.../gdrive/MyDrive/Brookings/XML_downloads/xml-operators-byaccountID/XML files/NO_101987_operator.xml
https://ec.europa.eu/clima/ets/exportEntry.do?form=ohaDetails&installationName=&permitIdentifier=&searchType=&mainActivityType=&accountType=&selectedPeriods=&complianceStatus=&languageCode=en&account.registryCode=NO&accountStatus=&accountID=101987&accountHolder=&registryCode=NO&installationIdentifier=&action=all&primaryAuthRep=&identifierInReg=&returnURL=installationName%3D%26accountHolder%3D%26search%3DSearch%26permitIdentifier%3D%26form%3Doha%26searchType%3Doha%26mainActivityType%3D-1%26currentSortSettings%3D%26installationIdentifier%3D%26languageCode%3Den&buttonAction=&exportType=1&exportAction=accountAll&exportOK=exportOK


In [ ]:
workingdir = "/gdrive/MyDrive/Brookings/XML_downloads/xml-operators-byaccountID/"
ID = int(6206)
getXMLbyAccountID(ID,"AT")

Downloading.../gdrive/MyDrive/Brookings/XML_downloads/xml-operators-byaccountID/XML files/AT_6206_operator.xml
https://ec.europa.eu/clima/ets/exportEntry.do?form=ohaDetails&installationName=&permitIdentifier=&searchType=&mainActivityType=&accountType=&selectedPeriods=&complianceStatus=&languageCode=en&account.registryCode=AT&accountStatus=&accountID=6206&accountHolder=&registryCode=AT&installationIdentifier=&action=all&primaryAuthRep=&identifierInReg=&returnURL=installationName%3D%26accountHolder%3D%26search%3DSearch%26permitIdentifier%3D%26form%3Doha%26searchType%3Doha%26mainActivityType%3D-1%26currentSortSettings%3D%26installationIdentifier%3D%26languageCode%3Den&buttonAction=&exportType=1&exportAction=accountAll&exportOK=exportOK
Not operator account!


Download all in loop...

In [ ]:
workingdir = "/gdrive/MyDrive/Brookings/XML_downloads/xml-operators-byaccountID/"
accounts = pd.read_csv(workingdir + "all_AccountIDs_Regcodes.csv")
accounts

,Account.ID,Registry.Code
0,6206,AT
1,7412,AT
2,6337,AT
3,13396,IT
4,13397,IT
...,...,...
33044,25120,SE
33045,74418,SI
33046,64364,SI
33047,65524,SI


In [ ]:
for i in range(0,len(accounts)):
# for i in range(0,len(accounts)):
  ID = int(accounts.loc[i,"Account.ID"])
  Reg = accounts.loc[i,"Registry.Code"]
  print(Reg+str(ID))
  try:
    getXMLbyAccountID(ID,Reg)
  except Exception as e:
    time.sleep(30)
    print("General exception, waiting 30 seconds")
    getXMLbyAccountID(ID,Reg)

Should run again in case those that failed on server did not get downloaded the second time. It should go quickly and just spit out "File already exists!"